In [1]:
import tensorflow as tf
import numpy as np
import models.mapping
import utils
import pickle
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
epochs = 100
patch_size = 384
batch_size = 8
keys = ["optical", "dem"]
n_outputs = 2

# Adjust model name to match with the intended model
model_name = "glavitu_regionencoding"

eps_diff = 1e-8
epochs_tolerance = 5

In [3]:
# Adjust the builder parameters manually if needed
model_builder = models.mapping.GlaViTU
model_args = {
    "input_shapes": {
        "optical": (patch_size, patch_size, 6),
        "dem": (patch_size, patch_size, 2),
    }, 
    "n_outputs": n_outputs, 
    "use_location": True,
    "location_size": 12,
    "dropout": 0.10, 
    "inference_dropout": False,
    "use_deepsupervision": True,
    "name": model_name,
}

In [4]:
weights_path = os.path.join("weights", f"{model_name}_weights.h5")
model = utils.build_model(
    model_builder, model_args,
    weights_path=weights_path, mode="testing"
)

2024-02-08 11:57:41.674658: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-08 11:57:41.711696: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-08 11:57:41.711888: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-08 11:57:41.712475: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the 

In [5]:
model.summary()

Model: "glavitu_regionencoding"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 location (InputLayer)          [(None, 12)]         0           []                               
                                                                                                  
 optical (InputLayer)           [(None, 384, 384, 6  0           []                               
                                )]                                                                
                                                                                                  
 dem (InputLayer)               [(None, 384, 384, 2  0           []                               
                                )]                                                                
                                                                             

In [6]:
# Set the path to the .pickle with the features
features_path = os.path.join("/path", "to", "file.pickle")
with open(features_path, "rb") as file:
    features, attrs = pickle.load(file)

In [8]:
batches_per_epoch = int(attrs["height"] * attrs["width"] / patch_size**2 / batch_size + 0.5)
batches_per_epoch

24

In [9]:
def DataGenerator(features, attrs, keys=keys, patch_size=patch_size, batch_size=batch_size):
    while True:
        batch = {key: [] for key in keys}
        height, width = attrs["height"],  attrs["width"]
        pad_height, pad_width = attrs["pad_height"], attrs["pad_width"]

        for _ in range(batch_size):
            row = np.random.choice(height - patch_size) + pad_height
            col = np.random.choice(width - patch_size) + pad_width
            row_slice = slice(row, row + patch_size)
            col_slice = slice(col, col + patch_size)
            for key in keys:
                patch = features[key][row_slice, col_slice, :]
                batch[key].append(patch)

        batch = {key: np.array(value) for key, value in batch.items()}
        yield batch

In [10]:
data_generator = DataGenerator(features, attrs)

In [12]:
def shannon_entropy(probs, eps=1e-6, C=float(n_outputs)):
    entropy = -tf.reduce_sum(probs * tf.math.log(probs + eps) / tf.math.log(C), axis=-1)
    return tf.reduce_mean(entropy)

In [13]:
optimised_bias = tf.Variable(tf.ones((12,)), trainable=True, dtype=tf.float32)
optimised_bias

<tf.Variable 'Variable:0' shape=(12,) dtype=float32, numpy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

In [15]:
epochs_with_eps_steps = 0

for _ in tqdm(range(epochs)):
    prev_bias = tf.Variable(optimised_bias)
    
    for _ in range(batches_per_epoch):
        batch = data_generator.__next__()
        with tf.GradientTape() as tape:
            tape.watch(optimised_bias)
            softmax_bias = tf.nn.softmax(optimised_bias)
            softmax_bias = tf.tile(tf.expand_dims(softmax_bias, axis=0), [batch_size, 1])
            batch["location"] = softmax_bias
            output = model(batch)
            loss = shannon_entropy(output)
        grads = tape.gradient(loss, [optimised_bias])
        optimizer.apply_gradients(zip(grads, [optimised_bias]))
        
    diff = tf.reduce_sum((optimised_bias - prev_bias) ** 2)
    if diff < eps_diff:
        epochs_with_eps_steps += 1
    else: 
        epochs_with_eps_steps = 0
        
    if epochs_with_eps_steps >= epochs_tolerance:
        print(f"{epochs_tolerance} with the bias difference less than {eps_diff}, early stopping.")
        break

  0%|                                                                         | 0/100 [00:00<?, ?it/s]2024-02-08 11:58:49.025071: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-02-08 11:58:49.687585: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2024-02-08 11:58:50.668475: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-08 11:58:50.669062: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-08 11:58:50.669087: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-02-08 11:58:50.669689: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-08 11:58:50.669756: W tensorfl

In [19]:
region_vector = np.array(tf.nn.softmax(optimised_bias))
region_vector

array([0.10099009, 0.06659123, 0.06655549, 0.07377534, 0.10282184,
       0.0664518 , 0.10387994, 0.06812785, 0.10622469, 0.10675227,
       0.0704275 , 0.067402  ], dtype=float32)

In [20]:
# Adjust the path where to save the optimised region vector to 
with open("region_vector.pickle", "wb") as dst:
    pickle.dump(region_vector, dst)